To copy this template: File -> Save a Copy in Drive

***DISCLAIMER**: In case of any discrepancy in the assignment instruction, please refer to the `PDF` document.*

# Problem 1 - Algorithmic Performance Scaling

## 1.1

**Link to dataset:**

In [1]:
# your code starts here
import openml
import pandas as pd
from openml.datasets import edit_dataset, fork_dataset, get_dataset

datalist = openml.datasets.list_datasets(output_format="dataframe")
datalist = datalist[(datalist['NumberOfInstances'] > 50000) & (datalist['NumberOfClasses'] > 2) & (datalist['NumberOfInstancesWithMissingValues'] == 0)]
# type(datalist)

!pip install tabloo
import tabloo
tabloo.show(datalist)

http://0624ab47fe24:40000/dtale/main/1

**Answer:**

## 1.2

In [ ]:
# your code starts here

**Answer:**

## 1.3

In [ ]:
# your code starts here

**Answer:**

## 1.4

In [ ]:
# your code starts here

**Answer:**

## 1.5

**Answer:**